## this code fixs main as simple as it can
```python
import ida_ua
import ida_name
import ida_bytes


def decode_instruction(ea: int) -> ida_ua.insn_t:
    instruction: ida_ua.insn_t = ida_ua.insn_t()
    instruction_length = ida_ua.decode_insn(instruction, ea)
    if not instruction_length:
        return None
    return instruction


def main():
    begin: int = ida_name.get_name_ea(idaapi.BADADDR, "start")
    end: int = begin + 0xE2

    instructions: dict[int, ida_ua.insn_t] = {}

    # Undefine the current code
    ida_bytes.del_items(begin, 0, end)

    # Follow the control flow and create instructions
    instruction_ea: int = begin
    while instruction_ea <= end:
        if instruction_ea not in instructions.keys():
            instruction: ida_ua.insn_t = ida_ua.insn_t()
            instruction_length: int = ida_ua.create_insn(instruction_ea, instruction)
        else:
            instruction: ida_ua.insn_t = decode_instruction(instruction_ea)
            instruction_length: int = instruction.size
        if not instruction_length:
            print(f"Failed to create an instruction at address {instruction_ea=:#x}")
            return

        # Append the current instruction address to the list
        instructions[instruction.ip] = instruction

        # Handle unconditional jumps
        current_instruction_mnemonic: str = instruction.get_canon_mnem()
        next_instruction: ida_ua.insn_t | None = decode_instruction(instruction_ea + instruction.size)
        if next_instruction is not None:
            next_instruction_mnemonic: str = next_instruction.get_canon_mnem()
            if (current_instruction_mnemonic == "jnz" and next_instruction_mnemonic == "jz") or \
                    (current_instruction_mnemonic == "jz" and next_instruction_mnemonic == "jnz"):
                # Unconditional jump detected
                assert instruction.ops[0].type == ida_ua.o_near
                instruction_ea = instruction.ops[0].addr

                ida_ua.create_insn(next_instruction.ip)
                instructions[next_instruction.ip] = next_instruction
                continue

        if current_instruction_mnemonic == "jmp":
            assert instruction.ops[0].type == ida_ua.o_near
            instruction_ea = instruction.ops[0].addr
        else:
            instruction_ea += instruction.size

    # NOP the remaining instructions
    for ea in range(begin, end):
        skip: bool = False
        for _, instruction in instructions.items():
            if ea in range(instruction.ip, instruction.ip + instruction.size):
                skip = True
                break
        if skip:
            continue

        # Patch the address
        ida_
```bytes.patch_bytes(ea, b"\x90")


if __name__ == "__main__":
    main()

## this code for function decryption stuff

```python
def xor_chunk(offset, n,key):
    ea = 0x400000 + offset
    for i in range(n):
        byte = ord(idc.get_bytes(ea+4, 1))
        bytekey0x50
        idc.patch_byte(a+i, 

xor_chunk(0x3292,0x2C,0x880BD3F6)
byte)

In [2]:
def xor_chunk(offset, n,key):
    ea = 0x400000 + offset
    for i in range(0,n,4):
        byte = idc.get_bytes(ea+i, 4)
        byte ^= key
        idaapi.patch_byte(ea+i,byte)


## this code decrypts API Hashes with respect to endianness

In [2]:
import pefile
import binascii
import struct
filepath=r"C:\Users\REM\Desktop\Threat Mon sample\smokeloader\Extracted files\5IH0Dp8.exe"
decrypted_file =r"C:\Users\REM\Desktop\Threat Mon sample\smokeloader\Extracted files\bin_LZSA2\decrypted_64_payload.bin"
pattern_64=b"f6810b88e1130b"
pattern_64=binascii.unhexlify(pattern_64)
x_64_size =0x2E46
pattern_86 =b'f6e90b88e1130b77'
pattern_86=binascii.unhexlify(pattern_86)
x_86_size=0x22F8
def get_encrypted_s3():
    pe=pefile.PE(filepath)
    for section in pe.sections:
        if b'text' in section.Name:
            return (section.get_data()[0x463a:0x463a+x_64_size],section.get_data()[0x2342:0x2342+ x_86_size])

data_64,data_86=get_encrypted_s3()
#print(data_86)
#unpacked = struct.unpack("<I",data_[0:4])[0]
#print(hex(unpacked))


In [3]:
def xor_chunk_API(offset, n, key, is_big_endian=False):
    ea = 0x400000 + offset
    for i in range(0, (n//4)*4, 4):
        # Get a chunk of 4 bytes
        chunk = idc.get_bytes(ea + i, 4)

        # Reverse byte order if using big-endian
        if is_big_endian:
            chunk = chunk[::-1]

        # Convert the bytes to an integer
        value = int.from_bytes(chunk, byteorder='little')

        # XOR the integer with the key
        xor_result = value ^ key

        # Convert the result back to bytes
        xor_bytes = xor_result.to_bytes(4, byteorder='little')

        # Patch the original bytes with the XOR result
        idc.patch_bytes(ea + i, xor_bytes)
    #X = n%4
    #if X > 0:
     #   byte_x = [ea
        
def xor_chunk_s3( data, dword_key, b_key):
    decrypted=b''
    #print(data)
    for i in range(0,(len(data)//4)*4,4):
        _4_bytes= struct.unpack("<I",data[i:i+4])[0]
        xor_result = _4_bytes ^ dword_key
        decrypted+=struct.pack("<I",xor_result)
    last_bytes_len = len(data)%4
    #print(last_bytes_len)
    #print()
    
    #print(decrypted)
    if last_bytes_len > 0:
        last_decrypted=[]
        for byte in data[-last_bytes_len:]:
            last_decrypted.append(byte ^ b_key)
            print(last_decrypted)
        #print(last_decrypted)
        decrypted+=bytes(last_decrypted)
    return decrypted
#print(f"len = {hex(len(data_64))} \n {binascii.hexlify(data_64)}   ")    
#print(hex((len(data_64)) >> 2))
#print(hex((len(data_64) // 4)*4))
#print(hex(x_64_size & 3))
#print((hex(x_64_size %4)))
#test_64 = xor_chunk_s3(data_64,0x880BD3F6,0xF6)
#size_64=hex(struct.unpack("<I",test_64[0:4])[0])
#print(f"size 64 = {size_64} and compressed length = {hex(len(test_64))}")
#print(binascii.hexlify(test_64))
test_86 = xor_chunk_s3(data_86,0x880BD3F6,0xF6)
size_86=hex(struct.unpack("<I",test_86[0:4])[0])
print(f"size 86 = {size_86} and compressed length = {hex(len(test_86))}")
#print(test_86)
payload = test_86[4:]
#print(payload)
#open(decrypted_file,'wb').write(payload)
#print("Written Done :) ")
#print(test_64)


size 86 = 0x3a00 and compressed length = 0x22f8
Written Done :) 


## decrpyting 3th stage Configuration 

In [ ]:
def rc4crypt(data, key):
    #If the input is a string convert to byte arrays
    if type(data) == str:
        data = data.encode('utf-8')
    if type(key) == str:
        key = key.encode('utf-8')
    x = 0
    box = list(range(256))
    for i in range(256):
        x = (x + box[i] + key[i % len(key)]) % 256
        box[i], box[x] = box[x], box[i]
    x = 0
    y = 0
    out = []
    for c in data:
        x = (x + 1) % 256
        y = (y + box[x]) % 256
        box[x], box[y] = box[y], box[x]
        out.append(c ^ box[(box[x] + box[y]) % 256])
    return bytes(out)
filepath=r"C:\Users\REM\Desktop\Threat Mon sample\smokeloader\Extracted files\bin_LZSA2\decompressed_86.bin"
dump = b'2d0c9bbe0672b7a2519cf8d4899033124f008c6bb854825507baab3d076ae7276985fcd584200e4df89c8467de25243780ac0276ecff1ba4dbcec485330e470a9d18835f855f19a2ab76494ef2323887e7d3840c058bbc1771e4be4cd6f2cbcb092880a91775e4e210c20b1483bf1168e3d20d91ecc206119caf0432bf08058bbc1771e4be4c06119da61b6ee3050b83af4533071386a41e75f9fd06139cf82932bf060081b91771e4071787af1a6dbebf071787a60160fde40a1799a92064fffe1797f807328ab80568e2e3044a8da30204419c8c300441dff80e0a419cef4639d5a84ec0ce0c41efb9765d8da87e9096d4a70841efb976248dfe7e1c16efaf76668dfe7e8e96d5a7c45c4f284ce96bca42f11a6be9ce71691241ef8b76518ddd7ebc96e6a7a35c3c2849e90c41ef9e76448dc07ea89682a7084aefaf76798de87e084aefae766d8de17e084aefa876608df97e205eef90766e8de37e9d9689a7be5c192809e92aca45f1536baace6b696e8a30540834ef8576528dd97e5e27efa5766f8df97e9d96c9a7835c502838e93dca41f15f6bf6ce22696a8a32549895daf6267d419e89aa70b12180ec1110e5b4528b785e1366c5dd547530a8d4f647d1ab1d6d6af25586f5ca7bfb7ddf1250858716b75b46ac95d0bc915b080befba76648de37e102cefa576728df97ec29687a7d25c0e280a34ef9e76308dbd7eb5962655eff376388db47ed59696a7c65c50285fe974ca65f10a6bfcce38693b8a7254d29586f67f7d4222efa376738de87e9e96c8a78f5c5d2828e921ca57f15b6bb9ce6e697f8a6254aa95c4f6207d559e9baa61b13a80a3113fe5fc5296781d1365c58a542730edd4e3474425efa976628de87e8896d3a7cd5c5d2846e96bca1bf1376bc6ce50696e8a24548d95c4f62a7d509ed2aa24b12080f7110ae5eb52c9785c133ec58f545130a0d4e34791ab4625efa976628de87e8896d3a7cd5c5d2846e96bca1bf1376bc6ce50696e8a24548d95c4f62a7d509ed2aa24b12080f7110ae5eb52807849133ec585542730d6d4b547cdab406d084aefa9766e8de07e084aefa576738dea7e084aefa476648df97e1801efb276718de17e9796d5a7925c0f2842e921ca49f15f6b'
dump= binascii.unhexlify(dump)
index = 0
key =0x246FC425
while index < len(dump):
    enc_length = str_data[index]
    x = rc4crypt(dump[index+1:index+1+enc_length], struct.pack('<I',key))
    print(x.replace(b'\x00',b'')) 
    index = index+1+enc_length